# Mise en place d'un algorithme de recherche de chemins sur les premiers exemples de vésicules


##### Importation des bibliothéques définis par l'utilisateur

In [1]:
import algoPSC2_0_1
import algoresolution_système

##### Lecture du fichier catalogue pour construire les listes de molécules et réactions

In [2]:
import re #bibliothéque python

file = "catalog.bc"
nb_réactions_max = 50
algoPSC2_0_1.nb_réactions_max=nb_réactions_max

elmts = []
inputs = []
enzymes = []
regex = ["(present\()|(, [e\-0-9]+\))|\.", "(MA.*for )|\+|(=>)|\."]
blocs = {"inputs": (regex[0], inputs), "enzymes": (regex[0], enzymes), "elmts": (regex[1], elmts)}

""" Création du tableau "reaction" de l'ensemble des réactions possible données par le fichier file"""
friends = {'test': []}  # Chaque elmt et l'ensemble des elements avec lesquels il reagit
# par contre les enzymes ne sont pas comptés comme éléments, du coup ils ne font pas partie des clés du dict.

reaction = []  # Liste des réactions en version texte
with open(file) as f:
    while f.readline() != "% Inputs\n":
        continue
    bloc = 'inputs'
    while True:
        l = f.readline()
        if not l:
            break

        if l[0] == '%':
            if l.find("Enzymes") > 0:
                bloc = 'enzymes'
                continue
            if l.find('reaction') > 0:
                bloc = 'elmts'
                continue
            continue

        if len(l) < 3:
            continue

        if bloc == 'elmts':
            elmt = l.split('=>')
            reactifs = re.sub("(MA.*for )|\+", " ", elmt[0]).split()
            for r in reactifs:
                try:
                    friends[r].extend([e for e in reactifs if e not in friends[r]])
                except:
                    friends.update({r: [e for e in reactifs if e != r]})
            produits = re.sub("\+|\.", " ", elmt[1]).split()
            reaction.append((reactifs, produits))
            elts = reactifs + produits
        else:
            elts = re.sub(blocs[bloc][0], '', l).split()

        blocs[bloc][1].extend([e for e in elts if (e not in blocs[bloc][1])])
algoPSC2_0_1.reaction=reaction

MOL = enzymes + elmts
MOL.pop(26)  # ?
algoPSC2_0_1.MOL=MOL


REACTIONS = []  # Liste des réactions en version numéros
for a in reaction:
    reac = ([], [])
    for m in a[0]:
        if m == 'H_20_2' or m == 'H202' or m == 'H_2O_2':
            reac[0].append(algoPSC2_0_1.numero('H2O2'))
        else:
            reac[0].append(algoPSC2_0_1.numero(m))
    for m in a[1]:
        if m == 'H_20_2' or m == 'H202' or m == 'H_2O_2':
            reac[1].append(algoPSC2_0_1.numero('H2O2'))
        else:
            reac[1].append(algoPSC2_0_1.numero(m))
    REACTIONS.append(reac)
algoPSC2_0_1.REACTIONS=REACTIONS


REACPARMOL = [] # REACPARMOL = ?
for k in range(0, len(MOL)):
    REACPARMOL.append([])
for k in range(0, len(REACTIONS)):
    u = REACTIONS[k][0][0]
    REACPARMOL[u].append(k)
algoPSC2_0_1.REACPARMOL=REACPARMOL

## Résolution d'exemples

In [3]:
##glucose et acetone donnent gluconolacrone
algoPSC2_0_1.ENZ = ['AO', 'ADH', 'G_1DH', 'NAD', 'resazurin', 'HRP', 'H2O2']  ##rajouter NAD pour fausser le résultat
MECAS = algoPSC2_0_1.résolution_équation("acetoneext + glucoseext => resorufin")


Les mécanismes après sélection sont les suivants
[([[6], [8], [10]], 'e', [5, 77, 4]), ([[24, 0, 1], [26], [28], [18], [20], [22], [23], [14], [16], [6], [8], [10]], 'ab', [5, 6, 1, 10, 3, 13, 15, 4])]
 
mécanisme 1:
[(['HRP', 'H_2O_2'], ['CCia5'])]
[(['CCia5', 'resazurin'], ['CCib5'])]
[(['CCib5'], ['HRP', 'resorufin'])]
 
mécanisme 2:
[(['G_1DH', 'NAD'], ['CCia1']), (['glucoseext'], ['glucose']), (['acetoneext'], ['acetone'])]
[(['CCia1', 'glucose'], ['CCib1'])]
[(['CCib1'], ['CCfa1', 'NADH'])]
[(['ADH', 'NADH'], ['CCia2'])]
[(['CCia2', 'acetone'], ['CCib2'])]
[(['CCib2'], ['CCfa2', 'NAD'])]
[(['CCfa2'], ['ADH', 'isopropanol'])]
[(['AO', 'isopropanol'], ['CCf3'])]
[(['CCf3'], ['CCio3', 'H_2O_2'])]
[(['HRP', 'H_2O_2'], ['CCia5'])]
[(['CCia5', 'resazurin'], ['CCib5'])]
[(['CCib5'], ['HRP', 'resorufin'])]
 
********************************************
[(['HRP', 'H_2O_2'], ['CCia5'])]
[(['CCia5', 'resazurin'], ['CCib5'])]
[(['CCib5'], ['HRP', 'resorufin'])]




In [4]:
## Test lescture équation
ENZ=['ABTS','ADH', 'NADH', 'resazurin', 'HRP', 'AO', 'HRP2', 'POD', 'NR', 'G_1DH', 'O2', 'DAF','NAD'] ##rajouter NAD pour fausser le résultat
RE=['acetoneext','glucoseext']
re=algoPSC2_0_1.numero2(RE)
enz=algoPSC2_0_1.numero2(ENZ)
solution=algoresolution_système.ressystem([re],[algoPSC2_0_1.numero('gluconolacrone')],['ab'],20,enz)

mt = algoPSC2_0_1.mecatexte(solution[0])
for d in mt[0]:
    print(d)
print("")
print("")

Les mécanismes après sélection sont les suivants
[([[24, 0], [26], [28], [29]], 'b', [10, 3, 13]), ([[24, 0], [26], [28], [29]], 'b', [10, 3, 13]), ([[18, 1, 0], [20], [22], [24], [26], [28], [29]], 'ab', [10, 1, 2, 15, 13]), ([[18, 1, 0], [20], [22], [24], [26], [28], [29]], 'ab', [10, 1, 2, 15, 13])]
 
********************************************
(['CCib5'], ['HRP', 'resorufin'])
(['acetoneext'], ['acetone'])
(['Lactateext'], ['Lactate'])
(['CCf3'], ['AO', 'isopropanol'])
(['CCf4'], ['HRP2', 'NADN'])




In [5]:
##NO et glucose donnent gluconolacrone avec NO3 en réactif annexe
algoPSC2_0_1.ENZ = ['ABTS', 'ADH', 'resazurin', 'HRP', 'AO', 'HRP2', 'POD', 'NR', 'G_1DH', 'O2', 'DAF', 'NAD']
algoPSC2_0_1.résolution_équation("NO2 + glucoseext => DAFF")

Les mécanismes après sélection sont les suivants
[([[40, 24, 0, 40, 24, 0], [26, 26], [28, 28], [44, 44], [48, 48], [31], [33], [34], [36], [37], [39]], 'ab', [12, 9, 24, 10, 3, 13, 11, 9, 24, 10, 3, 13])]
 
mécanisme 1:
[(['NR', 'NO2'], ['Cia3']), (['G_1DH', 'NAD'], ['CCia1']), (['glucoseext'], ['glucose']), (['NR', 'NO2'], ['Cia3']), (['G_1DH', 'NAD'], ['CCia1']), (['glucoseext'], ['glucose'])]
[(['CCia1', 'glucose'], ['CCib1']), (['CCia1', 'glucose'], ['CCib1'])]
[(['CCib1'], ['CCfa1', 'NADH']), (['CCib1'], ['CCfa1', 'NADH'])]
[(['Cia3', 'NADH'], ['Cfa3']), (['Cia3', 'NADH'], ['Cfa3'])]
[(['Cfa3'], ['Cio3', 'NO']), (['Cfa3'], ['Cio3', 'NO'])]
[(['O2', 'NO'], ['Cf6'])]
[(['Cf6'], ['O2', 'NO2b'])]
[(['NO', 'NO2b'], ['Cf5'])]
[(['Cf5'], ['N2O3'])]
[(['DAF', 'N2O3'], ['Cf4'])]
[(['Cf4'], ['DAFF'])]
 
********************************************
[(['NR', 'NO2'], ['Cia3']), (['G_1DH', 'NAD'], ['CCia1']), (['glucoseext'], ['glucose']), (['NR', 'NO2'], ['Cia3']), (['G_1DH', 'NAD'], ['CCia1'

In [6]:
## Ancien OU logique à changer sur le fonctionnement pluscourtchemin

ENZ=['ABTS', 'ADH', 'NAD', 'resazurin', 'HRP', 'AO', 'HRP2', 'POD', 'NR', 'G_1DH', 'O2', 'DAF', 'LO']
algoPSC2_0_1.ENZ = ENZ
RE = ['Lactateext', 'EtOHext']

algoPSC2_0_1.résolution_équation("Lactateext + EtOHext => ABTSOX")

MECAS = algoPSC2_0_1.pluscourtchemin(algoPSC2_0_1.numero2(ENZ), algoPSC2_0_1.numero2(RE), algoPSC2_0_1.numero('ABTSOX'), nb_réactions_max, True)  # Pourquoi tag a ? OU logique ?
mt = algoPSC2_0_1.mecatexte(MECAS[0][0])
for d in mt:
    print(d)

Les mécanismes après sélection sont les suivants
[([[84, 2], [69], [71], [86], [88]], 'a', [8, 0, 74, 17])]
 
mécanisme 1:
[(['POD', 'ABTS'], ['DDia5']), (['Lactateext'], ['Lactate'])]
[(['LO', 'Lactate'], ['DDf2'])]
[(['DDf2'], ['DDio2', 'H2O2'])]
[(['DDia5', 'H2O2'], ['DDib5'])]
[(['DDib5'], ['POD', 'ABTSOX'])]
 
********************************************
[(['POD', 'ABTS'], ['DDia5']), (['Lactateext'], ['Lactate'])]
[(['LO', 'Lactate'], ['DDf2'])]
[(['DDf2'], ['DDio2', 'H2O2'])]
[(['DDia5', 'H2O2'], ['DDib5'])]
[(['DDib5'], ['POD', 'ABTSOX'])]


Les mécanismes après sélection sont les suivants
[([[84, 2], [69], [71], [86], [88]], 'a', [8, 0, 74, 17])]
 
mécanisme 1:
[(['POD', 'ABTS'], ['DDia5']), (['Lactateext'], ['Lactate'])]
[(['LO', 'Lactate'], ['DDf2'])]
[(['DDf2'], ['DDio2', 'H2O2'])]
[(['DDia5', 'H2O2'], ['DDib5'])]
[(['DDib5'], ['POD', 'ABTSOX'])]
 
********************************************
[(['POD', 'ABTS'], ['DDia5']), (['Lactateext'], ['Lactate'])]
[(['LO', 'Lactate'],

### Test du non

In [7]:
##glucose et Non(acetone) donnent gluconolacrone
algoPSC2_0_1.ENZ = ['AO', 'ADH', 'G_1DH', 'NAD', 'resazurin', 'HRP', 'H2O2']  ##rajouter NAD pour fausser le résultat
algoPSC2_0_1.résolution_équation("!acetoneext + glucoseext => NADH")

Molécule non :  16

Les mécanismes après sélection sont les suivants
[([[24, 0], [26], [28]], 'b', [10, 3, 13]), ([[24, 0], [26], [28]], 'b', [10, 3, 13])]
 
mécanisme 1:
[(['G_1DH', 'NAD'], ['CCia1']), (['glucoseext'], ['glucose'])]
[(['CCia1', 'glucose'], ['CCib1'])]
[(['CCib1'], ['CCfa1', 'NADH'])]
 
mécanisme 2:
[(['G_1DH', 'NAD'], ['CCia1']), (['glucoseext'], ['glucose'])]
[(['CCia1', 'glucose'], ['CCib1'])]
[(['CCib1'], ['CCfa1', 'NADH'])]
 
********************************************
[(['G_1DH', 'NAD'], ['CCia1']), (['glucoseext'], ['glucose'])]
[(['CCia1', 'glucose'], ['CCib1'])]
[(['CCib1'], ['CCfa1', 'NADH'])]




In [8]:
##glucose et acetone donnent gluconolacrone
algoPSC2_0_1.ENZ = ['AO', 'ADH', 'G_1DH', 'NAD', 'resazurin', 'HRP', 'H2O2']  ##rajouter NAD pour fausser le résultat
algoPSC2_0_1.résolution_équation("acetoneext + glucoseext => resorufin")

Les mécanismes après sélection sont les suivants
[([[6], [8], [10]], 'e', [5, 77, 4]), ([[24, 0, 1], [26], [28], [18], [20], [22], [23], [14], [16], [6], [8], [10]], 'ab', [5, 6, 1, 10, 3, 13, 15, 4])]
 
mécanisme 1:
[(['HRP', 'H_2O_2'], ['CCia5'])]
[(['CCia5', 'resazurin'], ['CCib5'])]
[(['CCib5'], ['HRP', 'resorufin'])]
 
mécanisme 2:
[(['G_1DH', 'NAD'], ['CCia1']), (['glucoseext'], ['glucose']), (['acetoneext'], ['acetone'])]
[(['CCia1', 'glucose'], ['CCib1'])]
[(['CCib1'], ['CCfa1', 'NADH'])]
[(['ADH', 'NADH'], ['CCia2'])]
[(['CCia2', 'acetone'], ['CCib2'])]
[(['CCib2'], ['CCfa2', 'NAD'])]
[(['CCfa2'], ['ADH', 'isopropanol'])]
[(['AO', 'isopropanol'], ['CCf3'])]
[(['CCf3'], ['CCio3', 'H_2O_2'])]
[(['HRP', 'H_2O_2'], ['CCia5'])]
[(['CCia5', 'resazurin'], ['CCib5'])]
[(['CCib5'], ['HRP', 'resorufin'])]
 
********************************************
[(['HRP', 'H_2O_2'], ['CCia5'])]
[(['CCia5', 'resazurin'], ['CCib5'])]
[(['CCib5'], ['HRP', 'resorufin'])]


